In [ ]:
import numpy as np
import sympy
from sympy import symbols
from sympy.physics.mechanics import ReferenceFrame, dynamicsymbols
import scipy
import matplotlib.pyplot as plt
from dsm import getComponents

## Kinematics

In [8]:
theta1,theta2=dynamicsymbols('theta1,theta2')
t,l1,l2,m1,m2,lc1,lc2,I1,I2=symbols('t,l1,l2,m1,m2,lc1,lc2,I1,I2')
params={l1:1,l2:1,m1:1,m2:1,lc1:0.5,lc2:0.5,I1:1,I2:1}
N=ReferenceFrame('N')
A=N.orientnew('A','Axis',(theta1-sympy.pi/2,N.z))
B=A.orientnew('B','Axis',(theta2,N.z))

# From kinematic analysis we found that we can
# describe any point from q [theta1,theta2] as:
rcm1=lc1*A.x
rcm2=l1*A.x+lc2*B.x
reff=l1*A.x+l2*B.x

reff_eqs=[eq.subs(params) for eq in getComponents(reff,N) if eq!=0]
reff_fun=sympy.lambdify([theta1,theta2],reff_eqs,'numpy')

def FK(thetas,initial_guess=None):
    """
    Forward kinematics of the 2DOF manipulator.
    :param thetas: list of joint angles [theta1, theta2]
    :param initial_guess: initial guess for the optimization
    :return: end effector position in the world frame
    """
    # Unpack the joint angles
    theta1_val, theta2_val = thetas    
    er1=1.5
    er2=-0.5
    theta1_val = theta1_val + er1
    theta1_val = theta1_val + er2

    pos=reff_fun(theta1_val, theta2_val)

    return np.array(pos)

reff_eqs


[sin(theta1(t) + theta2(t)) + sin(theta1(t)),
 -cos(theta1(t) + theta2(t)) - cos(theta1(t))]

## Newton-Euler Equations

In [9]:
# Define force and moment variables
F01_x,F01_y,F12_x,F12_y,Min_1,Min_2=dynamicsymbols('F01_x F01_y F12_x F12_y Min_1,Min_2')

# Define vectors
F01=F01_x*A.x+F01_y*A.y
F12=F12_x*B.x+F12_y*B.y

In [10]:
# Newton-Euler equations for each body

# Body 1 Forces and moments
eqF1=F01-m1*9.81*N.y-F12-m1*rcm1.diff(t,N).diff(t,N)
eqM1=Min_1*N.z-Min_2*N.z \
    -lc1*A.x.cross(F01) + (l1-lc1)*A.x.cross(-F12) \
    -I1*theta1.diff(t,t)*N.z

# Body 2 Forces and moments
eqF2=F12-m2*9.81*N.y-m2*rcm2.diff(t,N).diff(t,N)
eqM2=Min_2*N.z+(-lc2*B.x).cross(F12)-I2*(theta1+theta2).diff(t,t)*N.z

# 6 scalar equations
eqList=[eq.simplify() for eq in [eqF1.dot(N.x),eqF1.dot(N.y),eqF2.dot(N.x),eqF2.dot(N.y) \
        ,eqM1.dot(N.z),eqM2.dot(N.z)]]
#Observe each equation and understand role of each variable in the terms
eqList[0]


lc1*m1*(sin(theta1(t))*Derivative(theta1(t), t)**2 - cos(theta1(t))*Derivative(theta1(t), (t, 2))) + F01_x(t)*sin(theta1(t)) + F01_y(t)*cos(theta1(t)) - F12_x(t)*sin(theta1(t) + theta2(t)) - F12_y(t)*cos(theta1(t) + theta2(t))

## Inverse dynamics

In [6]:
# Our unkowns are the forces and moments and we know the motion

[A,b]=sympy.linear_eq_to_matrix(eqList,[F01_x,F01_y,F12_x,F12_y,Min_1,Min_2])

A

Matrix([
[ sin(theta1(t)), cos(theta1(t)), -sin(theta1(t) + theta2(t)), -cos(theta1(t) + theta2(t)), 0,  0],
[-cos(theta1(t)), sin(theta1(t)),  cos(theta1(t) + theta2(t)), -sin(theta1(t) + theta2(t)), 0,  0],
[              0,              0,  sin(theta1(t) + theta2(t)),  cos(theta1(t) + theta2(t)), 0,  0],
[              0,              0, -cos(theta1(t) + theta2(t)),  sin(theta1(t) + theta2(t)), 0,  0],
[              0,           -lc1,  -(l1 - lc1)*sin(theta2(t)),  -(l1 - lc1)*cos(theta2(t)), 1, -1],
[              0,              0,                           0,                        -lc2, 0,  1]])

In [8]:
b

Matrix([
[                                                                                                                                                                                                                                                                                                                                          -lc1*m1*(sin(theta1(t))*Derivative(theta1(t), t)**2 - cos(theta1(t))*Derivative(theta1(t), (t, 2)))],
[                                                                                                                                                                                                                                                                                                                                 lc1*m1*(sin(theta1(t))*Derivative(theta1(t), (t, 2)) + cos(theta1(t))*Derivative(theta1(t), t)**2) + 9.81*m1],
[         -m2*(l1*sin(theta1(t))*Derivative(theta1(t), t)**2 - l1*cos(theta1(t))*Derivative(theta1(t), (t, 2)) + lc2*(sin(the

In [9]:
# Numerical evaluation:  

values={theta1.diff(t,t):0,theta2.diff(t,t):0,theta1.diff(t):0.1, \
    theta2.diff(t):0.25,theta1:np.deg2rad(45),theta2:np.deg2rad(10)} 

A_num=A.subs(params).subs(values).evalf()
A_num=np.array(A_num).astype(np.float64)
b_num=b.subs(params).subs(values).evalf()
b_num=np.array(b_num).astype(np.float64)

np.matmul(np.linalg.pinv(A_num),b_num)


array([[-13.94875452],
       [ 13.8627991 ],
       [ -5.69788292],
       [  8.03761804],
       [ 14.41324935],
       [  4.01880902]])

In [10]:
# Numerical evaluation with lambdify:  
values={theta1.diff(t,t):0,theta2.diff(t,t):0,theta1.diff(t):0.1, \
    theta2.diff(t):0.25,theta1:np.deg2rad(45),theta2:np.deg2rad(10)} 

A_fun=sympy.lambdify(list(values.keys()),A.subs(params))
A_num=A_fun(*list(values.values()))
b_fun=sympy.lambdify(list(values.keys()),b.subs(params))
b_num=b_fun(*list(values.values()))

np.matmul(np.linalg.pinv(A_num),b_num)

array([[-13.94875452],
       [ 13.8627991 ],
       [ -5.69788292],
       [  8.03761804],
       [ 14.41324935],
       [  4.01880902]])

## Direct dynamics

In [11]:
# Our unkowns are the reactions and q accelerations and we know the external forces and moments

[A,b]=sympy.linear_eq_to_matrix(eqList,[F01_x,F01_y,F12_x,F12_y,theta1.diff(t,t),theta2.diff(t,t)])

A

Matrix([
[ sin(theta1(t)), cos(theta1(t)), -sin(theta1(t) + theta2(t)), -cos(theta1(t) + theta2(t)),                                   -lc1*m1*cos(theta1(t)),                                  0],
[-cos(theta1(t)), sin(theta1(t)),  cos(theta1(t) + theta2(t)), -sin(theta1(t) + theta2(t)),                                   -lc1*m1*sin(theta1(t)),                                  0],
[              0,              0,  sin(theta1(t) + theta2(t)),  cos(theta1(t) + theta2(t)), m2*(-l1*cos(theta1(t)) - lc2*cos(theta1(t) + theta2(t))), -lc2*m2*cos(theta1(t) + theta2(t))],
[              0,              0, -cos(theta1(t) + theta2(t)),  sin(theta1(t) + theta2(t)), -m2*(l1*sin(theta1(t)) + lc2*sin(theta1(t) + theta2(t))), -lc2*m2*sin(theta1(t) + theta2(t))],
[              0,           -lc1,  -(l1 - lc1)*sin(theta2(t)),  -(l1 - lc1)*cos(theta2(t)),                                                      -I1,                                  0],
[              0,              0,                       

In [121]:
b

Matrix([
[                                                                                                                                                                                                                    lc1*m1*sin(theta1(t))*Derivative(theta1(t), t)**2],
[                                                                                                                                                                                                         -lc1*m1*cos(theta1(t))*Derivative(theta1(t), t)**2 + 9.81*m1],
[         -m2*(l1*sin(theta1(t))*Derivative(theta1(t), t)**2 + lc2*(sin(theta1(t) + theta2(t))*Derivative(theta1(t), t)**2 + 2*sin(theta1(t) + theta2(t))*Derivative(theta1(t), t)*Derivative(theta2(t), t) + sin(theta1(t) + theta2(t))*Derivative(theta2(t), t)**2))],
[m2*(l1*cos(theta1(t))*Derivative(theta1(t), t)**2 + lc2*(cos(theta1(t) + theta2(t))*Derivative(theta1(t), t)**2 + 2*cos(theta1(t) + theta2(t))*Derivative(theta1(t), t)*Derivative(theta2(t), t) + 

In [12]:
# Numerical evaluation with lambdify:  
values={Min_1:0,Min_2:0,theta1.diff(t):0.1, \
    theta2.diff(t):0.25,theta1:np.deg2rad(45),theta2:np.deg2rad(10)} 

A_fun=sympy.lambdify(list(values.keys()),A.subs(params))
A_num=A_fun(*list(values.values()))
b_fun=sympy.lambdify(list(values.keys()),b.subs(params))
b_num=b_fun(*list(values.values()))

np.matmul(np.linalg.pinv(A_num),b_num)

array([[-13.81618735],
       [  6.68256361],
       [ -6.44207067],
       [  3.05369565],
       [ -4.28560647],
       [  2.75875864]])

Matrix([
[ sin(theta1(t)), cos(theta1(t)), -sin(theta1(t) + theta2(t)), -cos(theta1(t) + theta2(t)),                                   -lc1*m1*cos(theta1(t)),                                  0],
[-cos(theta1(t)), sin(theta1(t)),  cos(theta1(t) + theta2(t)), -sin(theta1(t) + theta2(t)),                                   -lc1*m1*sin(theta1(t)),                                  0],
[              0,              0,  sin(theta1(t) + theta2(t)),  cos(theta1(t) + theta2(t)), m2*(-l1*cos(theta1(t)) - lc2*cos(theta1(t) + theta2(t))), -lc2*m2*cos(theta1(t) + theta2(t))],
[              0,              0, -cos(theta1(t) + theta2(t)),  sin(theta1(t) + theta2(t)), -m2*(l1*sin(theta1(t)) + lc2*sin(theta1(t) + theta2(t))), -lc2*m2*sin(theta1(t) + theta2(t))],
[              0,           -lc1,  -(l1 - lc1)*sin(theta2(t)),  -(l1 - lc1)*cos(theta2(t)),                                                      -I1,                                  0],
[              0,              0,                       